---
title: Water Rights Restored to the Gila River
subtitle: The impacts of irrigation on vegetation health in the Gila River Valley
author:
  - Elsa Culler
  - Nate Quarderer
date: last-modified
image: /img/earth-analytics/water-rights/lesson-water-rights.png
image-alt: "Dry river with dead plants turns into a stream with living plants"
description: |
  In 2004, the Akimel O'otham and Tohono O'odham tribes won a water rights settlement in the US Supreme Court. Using satellite imagery, we can see the effects of irrigation water on the local vegetation.
learning-goals:
- Open raster or image data using code
- Combine raster data and vector data to crop images to an area of interest
- Summarize raster values with stastics
- Analyze a time-series of raster images
params:
  id: stars
  site_name: Al Jawf Region Saudi Arabia
  project_name: Al Jawf Saudi Arabia Irrigation
  boundary_dir: 'al-jawf-region'
  event: water rights case
  start_year: '2001'
  end_year: '2022'
  event_year: '2012'
jupyter:
  kernelspec:
    name: learning-portal
    language: python
    display_name: Learning Portal
---




## Saudi Arabia is drilling for water

Groundwater irrigation has been growing in Saudi Arabia for the past 40 years. In this analysis, we'll observe the land-use changes brought on by drilling for water using satellite-based measurements.

::: {.callout-read}
[Desert Crops Thrive as the Aquifer Shrinks](https://earthobservatory.nasa.gov/images/145975/desert-crops-thrive-as-the-aquifer-shrinks)
:::


## Observing vegetation health from space

We will look at vegetation health using NDVI (Normalized Difference Vegetation Index). How does it work? First, we need to learn about spectral reflectance signatures.

Every object reflects some wavelengths of light more or less than others. We can see this with our eyes, since, for example, plants reflect a lot of green in the summer, and then as that green diminishes in the fall they look more yellow or orange. The image below shows spectral signatures for water, soil, and vegetation:

![](https://seos-project.eu/remotesensing/images/Reflexionskurven.jpg) \> Image source: [SEOS Project](https://seos-project.eu/remotesensing/remotesensing-c01-p06.html)

**Healthy vegetation** reflects a lot of **Near-InfraRed (NIR)** radiation. Less healthy vegetation reflects a similar amounts of the visible light spectra, but less NIR radiation. We don't see a huge drop in Green radiation until the plant is very stressed or dead. That means that NIR allows us to get ahead of what we can see with our eyes.

![Healthy leaves reflect a lot of NIR radiation compared to dead or stressed leaves](/img/earth-analytics/remote-sensing/spectral_vegetation_stress.png) \> Image source: [Spectral signature literature review by px39n](https://github.com/px39n/Awesome-Vegetation-Index)

Different species of plants reflect different spectral signatures, but the *pattern* of the signatures across species and sitations is similar. NDVI compares the amount of NIR reflectance to the amount of Red reflectance, thus accounting for many of the species differences and isolating the health of the plant. The formula for calculating NDVI is:

$$NDVI = \frac{(NIR - Red)}{(NIR + Red)}$$

::: {.callout-read}
Read more about NDVI and other vegetation indices:

  - [earthdatascience.org](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/vegetation-indices-in-python/calculate-NDVI-python/)
  - [USGS](https://www.usgs.gov/landsat-missions/landsat-surface-reflectance-derived-spectral-indices)

:::





# STEP 0: Set up

First, you can use the following parameters to change things about the workflow:


In [1]:
id = 'stars'
site_name = 'Al Jawf Region Saudi Arabia'
project_name = 'Al Jawf Saudi Arabia Irrigation'
boundary_dir = 'al-jawf-region'
event = 'water rights case'
start_year = '2001'
end_year = '2022'
event_year = '2012'

## Import libraries

We'll need some Python libraries to complete this workflow.

::: {.callout-task title="Import necessary libraries"}
In the cell below, making sure to keep the packages in order, add packages for:

  - Working with DataFrames
  - Working with GeoDataFrames
  - Making interactive plots of tabular and vector data

:::

::: {.callout-respond}
What are we using the rest of these packages for? See if you can figure it out as you complete the notebook.
:::


In [ ]:
#| echo: true
#| eval: false
import json
from glob import glob

import earthpy
import hvplot.xarray
import rioxarray as rxr
import xarray as xr

::: {.content-visible when-format="html"}


In [2]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
import json
from glob import glob

import earthpy
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import pandas as pd
import rioxarray as rxr
import xarray as xr

:::

## Download sample data

In this analysis, you'll need to download multiple data files to your computer rather than streaming them from the web. You'll need to set up a folder for the files, and while you're at it download the sample data there.

::: {.callout-caution title="GOTCHA ALERT!" icon="false"}
A lot of times in Python we say "directory" to mean a "folder" on your computer. The two words mean the same thing in this context.
:::

::: {.callout-task}
In the cell below, replace 'Project Name' with '{{< meta params.project_name >}} and 'my-data-folder' with a **descriptive** directory name.
:::


In [ ]:
#| echo: true
#| eval: false
project = earthpy.Project(
    'Project Name, dirname='my-data-folder')
project.get_data()

::: {.content-visible when-format="html"}


In [3]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
project = earthpy.Project(project_name)
project.get_data()


**Final Configuration Loaded:**
{}
🔄 Fetching metadata for article 29521241...
✅ Found 2 files for download.
[('https://ndownloader.figshare.com/files/56105732', 'al-jawf-ndvi', 'zip'), ('https://ndownloader.figshare.com/files/56105726', 'al-jawf-region', 'zip')]
al-jawf-ndvi
[PosixPath('/Users/elsa/Library/Application Support/earth-analytics/al-jawf-saudi-arabia-irrigation/al-jawf-ndvi')]
al-jawf-region
[PosixPath('/Users/elsa/Library/Application Support/earth-analytics/al-jawf-saudi-arabia-irrigation/al-jawf-ndvi'), PosixPath('/Users/elsa/Library/Application Support/earth-analytics/al-jawf-saudi-arabia-irrigation/al-jawf-region')]


:::


# STEP 1: Site map

## Study Area: {{< meta params.site_name >}}

### Earth Data Science data formats

In Earth Data Science, we get data in three main formats:

| Data type | Descriptions | Common file formats | Python type |
|---|---|---|---|
| Time Series | The same data points (e.g. streamflow) collected multiple times over time | Tabular formats (e.g. .csv, or .xlsx) | pandas DataFrame |
| Vector | Points, lines, and areas (with coordinates) | Shapefile (often an archive like a `.zip` file because a Shapefile is actually a collection of at least 3 files) | geopandas GeoDataFrame |
| Raster | Evenly spaced spatial grid (with coordinates) | GeoTIFF (`.tif`), NetCDF (`.nc`), HDF (`.hdf`) | rioxarray DataArray |

::: {.callout-read}
Check out the sections about about [vector data](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-vector-data-python/spatial-data-vector-shapefiles/) and [raster data](https://www.earthdatascience.org/courses/intro-to-earth-data-science/file-formats/use-spatial-data/use-raster-data/) in the textbook.
:::



::: {.callout-respond}
For this coding challenge, we are interested in the boundary of the {{< meta params.site_name >}}, and the health of vegetation in the area measured on a scale from -1 to 1. In the cell below, answer the following question: **What data type do you think the boundary will be? What about the vegetation health?**
:::

::: {.content-visible when-format="ipynb"}
:::: {.cell .markdown}
YOUR ANSWER HERE
::::
:::

### Load the {{< meta params.site_name >}} boundary

::: {.callout-task}

  - Locate the boundary files in your download directory
  - Change `'boundary-directory'` to the actual location
  - Load the data into Python and check that it worked

:::


In [ ]:
#| echo: true
#| eval: false
# Load in the boundary data
boundary_gdf = gpd.read_file(
    project.project_dir / 'boundary-directory')
# Check that it worked

::: {.content-visible when-format="html"}


In [4]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
# Load in the boundary data
boundary_gdf = gpd.read_file(
    project.project_dir / boundary_dir)
# Check that it worked
boundary_gdf

,element,id,admin_leve,boundary,name,name_en,type,ISO3166-2,alt_name_e,alt_name_f,...,name_vi,name_war,name_wuu,name_zh,place,population,populati_1,source_pop,wikidata,geometry
0,relation,3842543,4,administrative,منطقة الجوف,Al Jawf Region,boundary,SA-12,Al-Jawf Province,Al Jawf,...,Al Jawf,Al Jawf,焦夫省,焦夫省,state,440009,2010,stats.gov.sa,Q1471266,"POLYGON ((37.00617 31.50064, 37.01162 31.50183..."


:::


In [ ]:
#| echo: true
#| eval: false
# Plot the results with web tile images
boundary_gdf.hvplot()

::: {.content-visible when-format="html"}


In [5]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
# Plot the results with web tile images
boundary_gdf.hvplot(
    geo=True, tiles='EsriImagery',
    fill_color=None, line_color='black',
    title=site_name,
    frame_width=500)

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]

:::



# STEP 2: Wrangle Raster Data

## Load in NDVI data

Now you need to load all the downloaded files into Python. Let's start by getting all the file names. You will also need to extract the date from the filename. Check out [the lesson on getting information from filenames in the textbook](https://www.earthdatascience.org/courses/intro-to-earth-data-science/write-efficient-python-code/loops/data-workflows-with-loops/). 

Instead of writing out the names of all the files you want, you can use the `glob` utility to find all files that match a **pattern** formed with the **wildcard** character `*`. The wildcard can represent any string of alphanumeric characters. For example, the pattern `'file_*.tif'` will match the files `'file_1.tif'`, `'file_2.tiv'`, or even `'file_qeoiurghtfoqaegbn34pf.tif'`... but it will not match `'something-else.csv'` or even `'something-else.tif'`.

In this notebook, we'll use the `.rglob()`, or **recursive** glob method of the Path object instead. It works similarly, but you'll notice that we have to convert the results to a list with the `list()` function.

::: {.callout-caution title="GOTCHA ALERT!"}
`glob` doesn't necessarily find files in the order you would expect. Make sure to **sort** your file names like it says in the textbook.
:::

::: {.callout-respond}
Take a look at the file names for the NDVI files. What do you notice is the same for all the files? Keep in mind that for this analysis you only want to import the NDVI files, not the Quality files (which would be used to identify potential incorrect measurements).
:::

::: {.content-visible when-format="ipynb"}
:::: {.cell .markdown}
YOUR ANSWER HERE
::::
:::

::: {.callout-task}

  1. Create a **pattern** for the files you want to import. Your pattern should include the parts of the file names that are the same for all files, and replace the rest with the `*` character. Make sure to match the NDVI files, but not the Quality files!
  2. Replace `ndvi-pattern` with your pattern
  3. Run the code and make sure that you are getting all the files you want and none of the files you don't!
  
:::


In [ ]:
#| echo: true
#| eval: false
# Get a sorted list of NDVI tif file paths
ndvi_paths = sorted(list(project.project_dir.rglob('ndvi-pattern')))

# Display the first and last three files paths to check the pattern
ndvi_paths[:3], ndvi_paths[-3:]

::: {.content-visible when-format="html"}


In [6]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
# Get a sorted list of NDVI tif file paths
ndvi_paths = sorted(list(project.project_dir.rglob('*NDVI*.tif')))

# Display the first and last three files paths to check the pattern
ndvi_paths[:3], ndvi_paths[-3:]

([PosixPath('/Users/elsa/Library/Application Support/earth-analytics/al-jawf-saudi-arabia-irrigation/al-jawf-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2001145000000_aid0001.tif'),
  PosixPath('/Users/elsa/Library/Application Support/earth-analytics/al-jawf-saudi-arabia-irrigation/al-jawf-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2001161000000_aid0001.tif'),
  PosixPath('/Users/elsa/Library/Application Support/earth-analytics/al-jawf-saudi-arabia-irrigation/al-jawf-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2001177000000_aid0001.tif')],
 [PosixPath('/Users/elsa/Library/Application Support/earth-analytics/al-jawf-saudi-arabia-irrigation/al-jawf-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_days_NDVI_doy2020177000000_aid0001.tif'),
  PosixPath('/Users/elsa/Library/Application Support/earth-analytics/al-jawf-saudi-arabia-irrigation/al-jawf-ndvi/MOD13Q1.061_2001137_to_2022244/MOD13Q1.061__250m_16_d

:::

## Repeating tasks in Python

Now you should have a few dozen files! For each file, you need to:

  - Load the file in using the `rioxarray` library
  - Get the date from the file name
  - Add the date as a dimension coordinate
  - Give your data variable a name

You don't want to write out the code for each file! That's a recipe for copy pasta. Luckily, Python has tools for doing similar tasks repeatedly. In this case, you'll use one called a `for` loop.

There's some code below that uses a `for` loop in what is called an **accumulation pattern** to process each file. That means that you will save the results of your processing to a list each time you process the files, and then merge all the arrays in the list.

::: {.callout-task}

  - Look at the file names. How many characters from the end is the date? `doy_start` and `doy_end` are used to extract the day of the year (doy) from the file name. You will need to count characters from the end and change the values to get the right part of the file name. HINT: the index -1 in Python means the last value, -2 second-to-last, and so on.
  - Replace any required variable names with your chosen variable names

:::


In [7]:
#| echo: true
#| eval: false
doy_start = -1
doy_end = -1

# Loop through each NDVI image
ndvi_das = []
for ndvi_path in ndvi_paths:
    # Get date from file name

    # Open dataset

    # Add date dimension and clean up metadata
    da = da.assign_coords({'date': date})
    da = da.expand_dims({'date': 1})
    da.name = 'NDVI'

    # Prepare for concatenation

NameError: name 'da' is not defined

::: {.content-visible when-format="html"}


In [8]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
doy_start = -25
doy_end = -19

# Loop through each NDVI image
ndvi_das = []
for ndvi_path in ndvi_paths:
    # Get date from the file name
    doy = ndvi_path.name[doy_start:doy_end]
    date = pd.to_datetime(doy, format='%Y%j')

    # Open dataset
    da = rxr.open_rasterio(ndvi_path, mask_and_scale=True).squeeze()

    # Add date dimension and clean up metadata
    da = da.assign_coords({'date': date})
    da = da.expand_dims({'date': 1})
    da.name = 'NDVI'

    # Prepare for concatenation
    ndvi_das.append(da)

len(ndvi_das)

138

:::

## Combine Rasters
Next, stack your arrays by date into a time series using the `xr.combine_by_coords()` function. You will have to tell it which dimension you want to stack your data in.


In [ ]:
#| echo: true
#| eval: false
# Combine NDVI images from all dates

::: {.content-visible when-format="html"}


In [9]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
# Combine NDVI images from all dates
ndvi_da = xr.combine_by_coords(ndvi_das, coords=['date'])
ndvi_da

<xarray.Dataset> Size: 2GB
Dimensions:      (date: 138, y: 1622, x: 2397)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 19kB 37.01 37.01 37.01 37.01 ... 41.99 41.99 42.0
  * y            (y) float64 13kB 31.74 31.74 31.74 31.74 ... 28.37 28.37 28.37
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 1kB 2001-01-14 2001-01-16 ... 2020-01-20
Data variables:
    NDVI         (date, y, x) float32 2GB 0.1039 0.1047 0.1047 ... 0.1324 0.1261

:::


# STEP 3: Plot NDVI

::: {.callout-task title="Plot the change in NDVI spatially"}
Complete the following:

  - Select data from before the {{< meta params.event >}} ({{< meta params.start_year >}} to {{< meta params.event_year >}})
  - Take the temporal mean (over the **date**, not spatially)
  - Get the NDVI variable (should be a DataArray, not a Dataset)
  - Repeat for the data from after the {{< meta params.event >}} ({{< meta params.event_year >}} to {{< meta params.end_year >}})
  - Subtract the pre-event data **from** the post-event data
  - Plot the result using a **diverging** color map like `cmap=plt.cm.PiYG`

There are different types of color maps for different types of data. In this case, we want decreases to be a different color from increases, so we should use a **diverging** color map. Check out available colormaps in the [matplotlib documentation](https://matplotlib.org/stable/tutorials/colors/colormaps.html).
:::

::: {.callout-extra}
For an extra challenge, add the {{< meta params.site_name >}} boundary to the plot.
:::


In [ ]:
#| echo: true
#| eval: false
# Compute the difference in NDVI before and after

# Plot the difference
(
    ndvi_diff.hvplot(x='', y='', cmap='', geo=True)
    *
    gdf.hvplot(geo=True, fill_color=None, line_color='black')
)

::: {.content-visible when-format="html"}


In [ ]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
ndvi_diff = (
    ndvi_da
        .sel(date=slice(event_year, end_year))
        .mean('date')
        .NDVI 
   - ndvi_da
        .sel(date=slice(start_year, event_year))
        .mean('date')
        .NDVI
)
(
    ndvi_diff.hvplot(x='x', y='y', cmap='PiYG', geo=True)
    *
    boundary_gdf.hvplot(geo=True, fill_color=None, line_color='black')
)

:::


# STEP 4: Is the NDVI different after the {{< meta params.event >}}?

You will apply an NDVI threshold and determine how many pixels have healthy vegetation. You can then look at the growth of vegetation over time.

::: {.callout-task}

  * Apply an NDVI threshold to identify pixels with healthy vegetation
  * Check that your threshold worked
  
:::


In [ ]:
#| echo: true
#| eval: false
# Apply NDVI threshold

::: {.content-visible when-format="html"}


In [10]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
# Apply NDVI threshold
veg_da = ndvi_da.where(ndvi_da>0.3)
veg_da

<xarray.Dataset> Size: 2GB
Dimensions:      (date: 138, y: 1622, x: 2397)
Coordinates:
    band         int64 8B 1
  * x            (x) float64 19kB 37.01 37.01 37.01 37.01 ... 41.99 41.99 42.0
  * y            (y) float64 13kB 31.74 31.74 31.74 31.74 ... 28.37 28.37 28.37
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 1kB 2001-01-14 2001-01-16 ... 2020-01-20
Data variables:
    NDVI         (date, y, x) float32 2GB nan nan nan nan ... nan nan nan nan

:::


Now, add up the number of pixels with healthy vegetation plot that over time. What do you observe? Don't forget to write a headline and description of your plot!


In [ ]:
#| echo: true
#| eval: false
# Plot difference inside and outside the boundary

::: {.content-visible when-format="html"}


In [16]:
veg_da.groupby('date.year').count(dim=['x', 'y'])

<xarray.Dataset> Size: 2kB
Dimensions:      (date: 138)
Coordinates:
    band         int64 8B 1
    spatial_ref  int64 8B 0
  * date         (date) datetime64[ns] 1kB 2001-01-14 2001-01-16 ... 2020-01-20
Data variables:
    NDVI         (date) int64 1kB 6336 4694 4769 5909 ... 32925 30477 32339

In [ ]:
#| echo: true
#| eval: true
#| code-fold: true
#| code-summary: See our solution!
# Plot number of vegetated pixels over time
composite_da = (
    ndvi_da
    .groupby('date.year')
    .mean(dim=['date'])
)
veg_da = (
    composite_da
    .where(composite_da > 0.3, drop=True)
    .count(dim=['x', 'y'])
)
veg_df
display(veg_df)
(
    veg_df
    .NDVI
    .hvplot(
        title=f'Vegetation over time in {site_name}',
        ylabel='Vegetated pixels',
        xlabel='Year')
)

KeyError: "No variable named 'date.year'. Did you mean one of ('year',)?"

:::

::: {.content-visible when-format="ipynb"}
:::: {.cell .markdown}
YOUR HEADLINE AND DESCRIPTION HERE
::::
:::
